# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [11]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁
SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
# last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
count = 0
while True:
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    pieces = soup.find('div',{'class':'part_pictxt_3 lazyload'}).find_all('div',{'class':'piece clearfix'})
    if len(pieces) != count:
        count = len(pieces)
        print('網頁更新中...')
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight),10000;")
    else:
        print('到達頁面底端')
        break

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] IBM宣布新電池技術！海水萃取3種非重金屬原料　5分鐘可充電80%

▲美國科技大廠IBM。（圖／路透）
財經中心／台北報導
IBM宣稱已研發出一種新電池技術，3種原料可從海水萃取，是前所未有的組合，可以取代鎳跟鈷等重金屬，比現有的鋰電池更安全，更便宜，還可以在短短5分鐘內充電至80％的電池狀態。
IBM已和戴姆勒旗下的賓士研究部門、日本電池電解質供應商中央玻璃株式會社、以及電池製造商Sidus等公司合作，共同研發這款下一代電池設計，IBM說，儘管目前還處於探索階段，不過希望能夠實現這個研發計畫。
目前的大多數電池，都由重金屬製成，近年來，引發了不少污染環境的質疑，而且還有低薪聘僱非洲童工在鈷礦場工作的人道爭議，也因此，如果能有更先進的電池出現，勢必成為一大創新革命，而且有助於電動車的研發升級。
▲ 美國科技大廠IBM宣稱已研發出一種新電池技術，圖為IBM研究人員在IBM研究電池實驗室工作，結合和測試獨特的材料和配方（圖／翻攝自IBM官網）
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[1] 財政部射3箭迎資金回台　北富銀調查：逾75%客戶有意將資金匯回

▲北富銀自11月29日起於台北、台中、高雄一連舉辦4場「資金返鄉路 富邦領航行」巡迴講座，解析海外資金回台的專法及解釋令的規定、常見的疑難雜症。（圖／北富銀提供）
記者紀佳妘／台北報導
富邦金（2881）旗下台北富邦銀行最新調查發現，超過75%客戶有意將海外資金匯回台灣，且逾80%關切個人身分申請資金回台議題。北富銀對此指出，政府祭出專法優惠稅率鼓勵資金回台，磁力效應已發酵，近期諮詢需求明顯增多，這次與客戶面對面，疏通資金回台法令及實務卡關痛點，並針對回台資金提供個人化的規劃諮商。
全球反避稅政策浪潮，推升海外資金的稅務風險，不少人想藉助財政部「三支箭」，包括歡迎台商回台投資行動方案、發布解釋令、實施境外資金匯回管理運用及課稅條例，將資金匯回台灣。不過，北富銀表示，多數客戶反映，依循專法將資金匯回

記者林育綾／台北報導
位於台北的「空總台灣當代文化實驗場（C-Lab）」近來因空間規劃計畫引發外界誤解和爭議，藝文工作者、建築師紛紛關注這座占地7公頃的公共建設。《ETtoday新聞雲》特別邀請文化部長鄭麗君親自說明，「空總」究竟要發展成什麼樣的機構？定位和願景是什麼？它既不是美術館，也不是劇場，台灣社會能否想像和迎接一個「完全新型態的藝文機構」，與華山和松菸等場域一起加入21世紀首都的「文化創新廊帶」 。
▲文化部長鄭麗君親自說明「空總」的定位和願景。（圖／記者林敬旻攝）
找回遺棄的歷史記憶
▲空總過去2年籌備「軟體組織」，同時與社會對話。（圖／空總台灣當代文化實驗場提供）
空總與國際組織合作
空總過去2年便是在籌備「軟體組織」，同時與社會對話、與國際重要組織合作、簽署合作備忘錄，例如跟法國龐畢度所屬的IRCAM合設了亞洲第一個「聲響實驗室」，擁有最先進的49.4聲道、360度聲響硬體設備。
鄭麗君說明，雖是最先進的設備，創作卻有「在地思考」，例如最近一部創作是將白景瑞導演在60年代所拍攝的「無聲電影」《台北之晨》，透過採集當代台灣的聲音，再透過林強重新創作，跟過去60年代的影像做了一次對話，這樣的結合與火花，讓龐畢度中心的IRCAM藝術總監跟他們的工程師專業團隊都非常興奮，透過跟C-Lab的合作，跟亞洲文化相遇，這種普世價值的科技藝術，可以跟在地故事充分結合，舉出這個例子，便是她在這2年看見如此趨勢。
▲空總法國龐畢度的IRCAM，合設了亞洲第一個「聲響實驗室」，其一創作是將白景瑞導演在60年代所拍攝的無聲電影《台北之晨》，透過採集當代台灣的聲音，再透過林強重新創作，跟過去60年代的影像做了一次對話。（圖／記者林育綾攝）
8年計畫打造「國際級創新的藝文機構」
而空總今年11月也獲得行政院同意續辦，發展第二期計畫，將以8年為期邁向一個國際級創新的藝文機構，但這個大型的修整建計畫，在近日引發外界誤解及爭議。
她不否認這樣一個大型的計畫，需要很多時間來做規劃，因此並非如外界誤解的黑箱作業或即刻執行與動工，而是「會在接下來的2年內提出園區修整建計畫，讓行政院來審查核定；過程中的總預算及年度預算，也會依據文化部提報的新計畫併同報核，同時也希望能邀請建築界、都市計畫界、藝文界的所有朋友共同參與。」
對於外界質疑「僅有少數幾人決定」，鄭麗君鄭重澄清，「其實社會諮詢一直都在進行

▲AirPods Pro 延遲問題已獲改善。（圖／翻攝自9to5mac）
記者陳心怡／外電報導
與之前的AirPods相比，AirPods Pro做了許多音訊技術的改進，像是主動式降噪功能和通透模式。另一方面，還改善了藍牙延遲，軟體開發人員柯伊爾（Stephen Coyle）也發文探討這項改進。
柯伊爾指出，延遲對於影音播放而言並不是問題，因為影片播放可能會稍有延遲，以確保聲音同步。但是，延遲對於無法預測的聲音來說，是個問題，這包括用戶啟動的聲音、遊戲中的音效等等。
為了測量延遲，柯伊爾做了兩種不同的測試：
柯伊爾使用兩個軟體來播放要測量的聲音，一個是iOS鍵盤，另一個則是柯伊爾開發一款名為Tapt的遊戲。柯伊爾表示，這是一個很好的基準，因為自己在編寫遊戲時特別需要低音頻延遲，並且本身很熟悉其技術基礎。
柯伊爾開發的遊戲與iOS鍵盤之間的區別，在於前者可以啟動並持續保持有聲音，而後者則不能。借助柯伊爾他的應用程式和鍵盤，柯伊爾獲得一系列聲音。
對於設備和軟體的每種組合，柯伊爾都使用設定為90 bpm的節拍器連續觸發10種聲音，以確保聲音一致。至於iOS鍵盤，由於上述原因，柯伊爾放棄第一次敲擊，最後將這19個測量平均值，得出延遲時間數值。
測量出來的結果，使用蘋果的W1晶片的第一代AirPods測量的延遲為274毫秒，第二代AirPods由蘋果最新的H1晶片提供支援，延遲時間已降至178毫秒，而使用相同H1晶片的AirPods Pro則為144毫秒。
柯伊爾稱表示，在感知上的差異，使AirPods Pro變得非常接近無延遲。另外值得注意的是，儘管第二代AirPods和AirPods Pro都使用相同的H1晶片，但蘋果似乎找到了新的方法來改善AirPods Pro的延遲問題。
除了AirPods之外，Beats Studio 3的延遲時間剛好超過250毫秒，其次是 Sony WH-CH700N，延遲時間不到250毫秒。
▲耳機聲音延遲測試結果。（圖／翻攝自9to5mac）
----------------------------------------------------------------------

[18] 雲端最有錢／放下賠錢炸彈　2020飆股、轉機股搶先卡位戰

新聞節目中心／綜合報導
美股搶搶滾，那台股又是如何利多盡出呢？這一波美中貿易協定慶祝行情

▲合庫積極推動自動化服務，積極推動VTM、ATM無卡服務。（圖／記者吳靜君攝）
記者吳靜君／台北報導
自動櫃員機會做的事情愈來愈多，不少銀行也讓自動櫃員機具有「開戶」的功能，為了提升數位化，銀行自動化服務，合庫表示，近期已在營業部數位體驗區為示範點，推出遠程視訊櫃員機（VTM），開戶時間可以節省三成以上。
近期有銀行撤銷自動櫃員機的服務，引發市場討論，認為金融科技是否會漸漸取代自動櫃員機（ATM）的功能。但另一方面，不少銀行認為，金融科技短期內難以取代ATM的功能與需求，甚至還扮演了「分行的分身」角色，不斷增加自動櫃員機的功能，在分行逐漸萎縮之下，ATM更能深入服務金融需求者。
自動櫃員機會做的事情愈來愈多，而不少銀行也讓自動櫃員機具有「開戶」的功能。一般來說，自動櫃員機具開戶功能者分「VTM」和「STM」（智慧櫃員機）
VTM是民眾透過螢幕與遠端銀行員對話，並且依照指示完成開戶認證、上傳證件、資料填寫到發卡就可以完成，強調可以24小時開戶、不用趕銀行營業時間忙著開戶，VTM可以辦理的事物，除了開戶之外，還可以申貸信用卡、舊貸戶對保、財富管理的KYC等，部分VTM也可以存、提、匯款等作業服務；STM則是結合櫃員機與人工，所以相較於VTM，STM造價比較便宜，一台大約是數十萬元即可，但是因為必須要與人工結合，所以必須要設置在營業大廳，所以仍必須在銀行營業時間辦理，目前推STM以兆豐銀為主。
除了一銀、華銀和玉山銀有VTM之外，合庫也跟進推出。合庫表示，只要雙證件，即可使用VTM與視訊櫃員對話，並透過視訊櫃員之引導與協助，逐步完成確認身分、人像拍攝、資料輸入及密碼設置等開戶手續後即可立即取得金融卡，大幅縮短傳統櫃檯開戶作業時間，同時藉由SOP化的作業流程降低作業風險、提升作業效率。合庫表示，至少可精簡30%的開戶時間，未來將評估延長VTM營運時間，讓開戶服務不用趕三點半。
▲人工與櫃員機開戶比較表。（製表／記者吳靜君、資料來源／各銀行、以各銀行公告為主）
▼兆豐銀的智慧櫃員機(STM)。（圖／記者吳靜君攝）
無卡時代　ATM存提款免帶卡
在ATM強化上，合庫進一步提供指靜脈無卡提款及台灣Pay行動提款，包含QR Code掃碼提款、NFC感應提款服務，初期在全台設置21台指靜脈自動櫃員機，自開辦以來，目前累計使用指靜脈提款的戶數成長約達31％。
為滿足不同客群需求與提升使

記者洪巧藍／台北報導
永齡健康基金會今（21）日在台大癌症醫院舉辦「新醫療百年工程薪傳論壇」，創辦人郭台銘致詞表示，他給自己的70歲生日禮物是成立個人「健康大數據」，每天記錄飲食等健康資料，幫助達成活到120歲的目標。郭也透露，女兒妞妞更是從出生開始做健康大數據到現在已經10歲，有不少朋友來說親，他要求未來女婿也必須提供出生起的健康大數據作為八字來對，他笑稱「要娶我女兒很困難」。
▲郭台銘出席永齡健康基金會「新醫療百年工程薪傳論壇」會後受訪。（圖／記者呂佳賢攝）
目標讓癌症變成慢性病，永齡健康基金會與台大永齡健康研究院，今(21)日共同舉辦「新醫療百年工程薪傳論壇」，研討會中邀請國內外產學專家與會，並宣布與國際七大頂尖生技公司聯盟合作，聚焦於數據醫學、液態活檢、先進檢驗醫學等領域，持續滾動「新醫療百年工程」的願景。
郭台銘致詞時表示，癌症一直是人類重要天敵，對醫學界也是很大挑戰，他剛從美國回來，短短三天行程從矽谷、舊金山、洛杉磯看到聖地牙哥，發現美國近期興起健康科技產業發展，而未來癌症基因大數據應該和台灣ICT產業進行結合。
▲永齡健康基金會「新醫療百年工程薪傳論壇」與七大國際頂尖生技大廠簽署MOU。（圖／記者呂佳賢攝）
郭台銘認為，癌症醫院做研究應該往前溯源，癌症發生的原因是什麼？除了天生基因，和每天吃的食物、喝的水、呼吸的空氣有關。他透露，兩個月前過70歲生日給自己一個禮物，成立個人「健康大數據」，把每天飲食等相關資訊進行記錄，不是一年一次健檢，而是天天都健檢，這樣才有機會達成他「活到120歲」的目標。
郭台銘也自曝，大女兒妞妞從出生開始就做大數據，每天吃什麼、喝什麼甚至得什麼病、打什麼預防針，已經建立10年份的大數據。已經10歲的女兒現在也很多人來說親，他強調，要說親可以，但要那名男性從出生到結婚前所有的健康大數據當成八字來對。
▲郭台銘出席永齡健康基金會「新醫療百年工程薪傳論壇」。（圖／記者呂佳賢攝）
永齡健康基金會連結國內外頂尖醫療團隊，促成此次與七大國際頂尖生技大廠(Thermo Fisher Scientific、Guardant Health、Merck、Konica Minolta、NanoString Technologies、Becton, Dickinson and Company、Roche)簽署合作備忘錄的機會。
郭台銘表示，希望藉由此舉

▲東區商圈房東降店租了，房市達人分析「還是不夠」。（圖／東森新聞）
記者趙蔡州／綜合報導
曾面臨倒閉潮的東區商圈開始恢復元氣了，根據房仲業調查的最新資料顯示，2019年第四季東區網路待租店面，相較前一季已經減少了2成左右，原因則是房東們聯合調降租金、年跌幅達11.5%，只是有房市專家仍認為，如果想要拯救東區商圈，只降一成租金是不夠的。
實際走訪東區商圈可以看見，雖然還是有很多店面掛著招租牌子，但比較之前倒閉潮的清冷場景，現場已經有不少店家漸漸開業。當地的業者就說，「半年前的時候我們就有評估了，想說相較之下這裡的房租，沒有以前那麼高了」，另一名業者則說，「人潮雖然沒有以前這麼多，但是基本消費力都是非常足夠的。」
▲房東透過降店租來吸引商家進駐。（圖／東森新聞）
台北市東區商圈2019年第四季至12月中旬，網路待租店面有182間，比前一季減少2成創下今年新低，主要原因則是租金調降，待租店面平均每坪約5212元，年跌幅達到11.5%。房市專家胡偉良表示，「房東用降租吸引客層是對的，但我認為只有降一成，仍不足以恢復以前榮景，要恢復的話，可能要降4、5成。」
有民眾表示，「我知道這邊之前是飲料店，後來好像有荒廢一陣子，然後它又開了手機店，希望這個東區可以再活絡」。另一位民眾則說「最近來逛東區感覺就……很多都是收了這樣，又或是在裝潢，然後換別的這樣子，但感覺之前比較熱鬧啦。」
▲房市專家認為，如果想要拯救東區商圈，只降一成租金是不夠的。。（圖／東森新聞）
據悉，東區大安路一段店面單坪平均租金，從8千到1萬2 降至約5千多元，而敦化南路一段190巷周遭，也從年初時一坪6千降到4千左右。只是房市專家胡偉良認為，現在商圈已經開始分流，單一商圈很難有以前龐大的集客量了，加上客層的消費能力降低，東區想回到過去的輝煌，恐怕還有一大段路要走。
▼影片取自YouTube，如遭刪除請見諒。
----------------------------------------------------------------------

[51] 和億集團攜手大金祭好禮　跨界打造好氣場搶攻年底聚餐商機

▲和億餐飲集團攜手變頻空調領導品牌大金，推出空氣清淨機抽獎活動，搶攻聖誕、跨年與新春的聚餐商機。圖右為大金空調董事長蘇一仲與和億執行長蘇嬉螢。（圖／和億餐飲集團提供）
記者林淑慧／台北報導
跨界聯名策

最近，標普和彭博社分別把台灣今年的經濟成長率調升到2.5%，明年估計為2.4%和2.3%，都比之前的估計高了一些。另外，隨著主計總處把台灣經濟成長率由原先的2.46%調高到2.64%之後，前兩天中研院也估計今年台灣的經濟成長率達到2.62%。看起來，台灣今年的經濟表現應該要比年初時的估計的確要好一點。但是，2.64%的經濟成長率真的會好到讓人民有感嗎？另外，兩個重要的數據讓現在的政府一直說台灣的經濟是不錯的，一個是台股指數達到12000點，是二十多年來的新高；另外，台商回台投資的金額達到7000億新台幣，經濟部估計，未來將可以為台灣創造5萬多個就業機會。
但是，如果台灣的經濟真的很好，為什麼前不久人們選擇今年的代表字，前三名會是「亂、謊和憂」？這代表說台灣大多數人對於今年的經濟、政治和社會的表現，其實是很不滿意的，所以才會選出這麼一些負面的代表字。
其次，由於這些台商預計回台投資，因而拉高了今年的資本形成，所以也帶動了經濟成長。但是，如果我們看今年的進出口，其實今年都是負成長的；另一方面，由於受到陸客減少及軍公教退休金被砍的問題，台灣今年的觀光業是很慘的，因此台灣今年的國內消費也直接受到影響。在主計總處和中研院今年最新的估計中，台灣民間消費的成長率都不到2%，遠低於全年的經濟成長率，也就是說，今年其實大多數民眾是很辛苦的，因為他們消費的力道比去年要差很多。
在此，我們再提供一些庶民可能會比較有感的數據，讓大家看一下台灣真正的經濟情況：第一，去年台灣的公司解散家數比前一年高出30.87%，今前10個月又比去年同期多出28.65%，今年全台灣解散的公司很可能會超過4萬家！第二，受到美中貿易戰爭的影響，今年前十個月的出口訂單成長率為-5.83%，實際的出口成長率則為-2.40%%。同時，因為出口不順，使得今年前三季台灣製造業產值比去年同期少了7.0%。第三，由於生產和出口不順，使得國發會公布的景氣燈號已經連續十個月出現趨弱的「黃藍燈」，表示台灣的經濟是欲振乏力！第四，金管會公布今年前三季上市公司累計淨利為1.44兆，比去年同期少了19%！
總而言之，台灣現在是在經濟成長率、股市和投資看起來數據不錯，但是民眾和企業真正感受到的經濟情況，可能完全不一樣！我們建議大家有空到一些觀光點和夜市走走感受一下，除了周末的情況還可以外，非假日期間這些地方的景氣情況真的很糟，政府真的要再加

▲亞太電信董事長呂芳銘。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
台灣5G頻譜五家電信仍在如火如荼地廝殺投標當中，雖然結果尚未出爐，不過身為「二小」之一的亞太電早已看好工業、健康，以及交通等垂直領域應用。
亞太電信董事長呂芳銘指出，就目前看來，5G技術未來在工業、健康，以及交通等三大垂直應用領域當中發展得最好，亞太電也早在此之前就在這三大領域中有所佈局。至於在消費者端方面，5G用戶數目成長速度還是要端看未來應用端能否激起需求。
至於在先前吵得沸沸揚揚的「企業專頻」部分，呂芳銘相當看好相關政策拍板定論，因為近期台灣將會有一波新廠開始陸續建置，恰逢5G將開始正式商轉，電信業可協助企業抓穩這波智慧製造的商機。
----------------------------------------------------------------------

[68] 理財周刊／台股站穩一萬二　教你算出主力成本

Ｑ：12月13日川普宣布延緩對中國課徵關稅後，台股一飛沖天，創下29年新高，站穩12,000整數關卡，面對台積電、大立光等大型權值股領漲，投資人一面開心股市多頭，又深怕只是曇花一現。台股位階這麼高，是否還能買股票？有沒有什麼方式能快速知道主力成本，買在相對低點？
Ａ：中美貿易和戰協議及聯準會利率按兵不動，為國際動盪局勢帶來一絲曙光，美股三大指數同步創高，台股也沾光站上一萬二整數大關，就外資買盤來看，外資主流投資在台積電、大立光、聯發科等大型權值股，題材面則以半導體、蘋概股、5G概念股等為主流。
台灣電子業供應鏈成熟，上游概念股受青睞，下游個股也能順勢上漲，在外資沒有大量賣超權值股之前，後續台股都仍有上漲空間，操作上建議利用外資買超作為主要依據，選出外資連買標的，透過量能檢測，找到合適進場價位，筆者這邊以台積電舉例，說明如何透過量能找到主力成本。
一般來說當股價上漲時，通常會伴隨大量，而這根大量Ｋ線的開盤價通常是主力成本，多頭趨勢回後買上漲，當我們知道主力成本在哪時，就能夠買在相對低點，最簡單找到主力成本方式是長紅Ｋ線及跳空缺口，如附圖灰藍色框框所示，缺口處爆大量，這樣的缺口下次就會形成強力支撐。另一處則是長紅開盤處，長紅往上開高走高，這也是標準主力成本價，下次回到這兩個價格附近將會有強力支撐，也是還沒有進場朋友的機會，來到這個價格不妨可以分批布局。
整體來看，外資不滅

▲捷運廣告進入多屏時代，多元化的呈現方式，讓廣告質感大躍進，吸睛指數百分百，成功吸引民眾目光。（採訪撰稿／記者陳珊珊；攝影剪輯／記者謝婷婷）
記者陳珊珊／採訪報導
放假好無聊，想拍張有趣照片，上傳社群平台嗎？與三五好友相約，走進捷運忠孝復興站就對了！統一麥香出招，打造趣味十足的拍照看板，讓你輕鬆拍出創意梗照。ASUS電腦也再升級，全新美麗雙螢幕，讓你創意無界限。
走在捷運站，被唯美細緻的遊戲畫面吸住目光，《諸神學院Smash》擁有豐富多樣的挑戰與模式，不管你是初學者或是資深卡牌玩家，都能在神話冒險世界中輕鬆稱霸。
▲《諸神學院Smash》為了達到更強烈的宣傳，選擇在捷運刊登新視頻廣告。（圖／記者謝婷婷攝）
諸神學院廣告代理商總經理江緯倫說：「這一款《諸神學院Smash》在畫風上非常日系，所以我們希望可以有更多的內容呈現給對於這款遊戲還不認識的遊戲玩家，因此我們選擇這一塊新視頻，來展現更多動畫的內容，達到更強烈的宣傳」。
來到捷運台北小巨蛋站，大氣勢巨幅影音呈現眼前，強力攻佔所有人目光，性感女神「李毓芬」以甜美形象，陪伴玩家一起漫遊《美人剎》世界。來到小巨蛋聽演唱會，音樂愛好者絕對不可缺少LINE MUSIC！
▲捷運台北小巨蛋站，大氣勢巨幅影音呈現眼前，強力攻佔所有人目光。（圖／記者謝婷婷攝）
LINE內容事業部副總張景雯表示，「LINE MUSIC其實不只是一個音樂串流的服務，使用者甚至可以分享音樂，就像分享貼圖一樣，非常方便，所以我們希望能夠用一個比較顯眼、比較科技感的視頻來做宣傳，這也跟我們LINE的創新精神結合度非常高，所以我們首次就嘗試用大屏幕的宣傳方式」。
▲▼LINE MUSIC選擇以顯眼、科技感的大屏幕做宣傳，成功吸引民眾注意。（圖／記者謝婷婷攝）
張景雯說：「LINE非常希望能夠宣揚在地文化，所以這一次也特別跟風潮音樂，還有屏東縣泰武國小的古謠傳唱，我們跟他們合作一個非常有愛的LINE來電答鈴鈴聲，從12月20日開始，全台灣LINE的用戶，都可以把這首非常有愛的《祝你耶誕快樂》設成自己的來電答鈴」。　
年底聚會大爆滿，還不知道怎麼穿搭嗎？跟著男神吳尊的腳步準沒錯，或是交給「魔法衣櫥」幫你量身打造，搖身一變，成為最耀眼的焦點。在外奔波一整天，害怕把髒污都帶回家了嗎？捷運站內影音告訴你，交給易洗樂抗菌防蟎洗衣精就不用擔心。
▲▼新視頻廣告讓民眾在搭捷運

▲鴻海威州建廠施工現場。（圖／翻攝鴻海影片）
記者姚惠茹／綜合外電報導
威斯康辛州長艾佛斯（Tony Evers）在美國時間19日接受美聯社訪問表示，鴻海將威州廠改為中小尺寸液晶面板廠，而不是原本提案的大尺寸面板廠，如果鴻海能與威州重新修訂合約，反映實際建廠計畫，仍可符合最高30億美元的減稅優惠資格，對此，鴻海聲明正與威州政府磋商。
艾佛斯表示，鴻海威州廠與當初簽訂合約的狀況不同，因此需要重新修訂合約條款，如果修改後的合約能夠確實反映威州廠建廠計畫，鴻海仍可獲得減稅優惠，並稱雙方應該進行協商，相信明年初能敲定協議，對此，鴻海發出聲明證實，但並未詳述內容。
鴻海2017年與威州前任州長華克簽約，計劃在威州投資100億美元興建面板廠，聘僱1.3萬名員工，並稱這項計畫有望提振當地經濟，獲得威州及地方政府承諾將近40億美元稅務優惠，美國總統川普甚至曾讚許鴻海威州廠是「世界第八大奇蹟」。
----------------------------------------------------------------------

[86] 投資台灣正夯　經部：前11月僑外投資額年增近二成

▲經濟部投審會今（20）日發佈最新統計，今年前11月核准僑外投資金額達102億餘美元，較去年同期成長將近二成。（圖／記者湯興漢攝）
記者林淑慧／台北報導
中美貿易戰延燒，不只台商加快回台腳步，外商也看好投資台灣。經濟部投審會今（20）日發佈最新統計，今年前11月核准僑外投資金額達102億餘美元，較上年同期成長將近二成，包括台灣美光半導體、沃旭能源等大型風力發電商，均有加碼投資。
依據經濟部統計，今年1至11月核准僑外投資件數為3,698件，較上年同期增加10.62％；投(增)資金額計美金102億919萬8,000元，較上年同期增加19.97％。
官員指出，今年以來僑外投資案中，較大投資金額者包括：荷商Micron Technology B.V.以約21.23億美元增資台灣美光半導體股份有限公司；英屬維京群島商KOHL Holdings Ltd.(美國Alphabet集團)以約8.35億美元增資特許投資顧問股份有限公司；以及德商Yunlin Holding Gmbh增資允能風力發電、丹麥商Orsted Wind Power TW Holding A/S增資沃旭能源等大型風力發電投資，合計約1